In [3]:
import pandas as pd
import numpy as np
from plotnine import *

In [4]:
# Import Local Data
local = pd.read_csv('../Data/NTD-Funding-Sources-Local.csv')

# Standardize column names
local.columns = local.columns.str.strip().str.lower().str.replace(' ', '_')

local['total_funding'] = local['total']
# local.to_csv('Cleaned-NTD-Funding-Local.csv', index=False, na_rep='NA')

# Standardize column names
# expenses.columns = expenses.columns.str.strip().str.lower().str.replace(' ', '_')
# expenses.to_csv('Cleaned-NTD-Expenses.csv', index=False, na_rep='NA')

In [5]:
funding = pd.read_csv('../Data/Cleaned-NTD-Funding.csv')
expenses = pd.read_csv('../Data/Cleaned-NTD-Expenses.csv')

In [6]:
merge_keys = [
    'agency', 'city', 'state',
    'ntd_id', 'organization_type', 'reporter_type',
    'report_year', 'uace_code', 'uza_name',
    'primary_uza_population'
]
nta = pd.merge(funding, expenses, on=merge_keys, how='outer')

nta.to_csv('Merged-NTA.csv', index=False, na_rep='NA')

In [7]:
nta = pd.read_csv('../Data/Merged-NTA.csv')

nta['profit'] = nta['total_funding'] - nta['total_expenses']
nta.to_csv('Merged-NTA.csv', index=False, na_rep='NA')

In [8]:
nta.isnull().sum()

agency                                           0
city                                             0
state                                            0
ntd_id                                           0
organization_type                                0
reporter_type                                    0
report_year                                      0
uace_code                                       54
uza_name                                        54
primary_uza_population                          54
voms                                             0
general_fund                                     0
income_tax                                       0
sales_tax                                        0
property_tax                                     0
fuel_tax                                         0
other_tax                                        0
tolls                                            0
other_funds                                      0
reduced_funds                  

In [9]:
nta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 34 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   agency                                        132 non-null    object 
 1   city                                          132 non-null    object 
 2   state                                         132 non-null    object 
 3   ntd_id                                        132 non-null    int64  
 4   organization_type                             132 non-null    object 
 5   reporter_type                                 132 non-null    object 
 6   report_year                                   132 non-null    int64  
 7   uace_code                                     78 non-null     float64
 8   uza_name                                      78 non-null     object 
 9   primary_uza_population                        78 non-null     flo

### Replacing the missing values

In [10]:
nta['primary_uza_population'].fillna(nta['primary_uza_population'].mean(), inplace=True)

C:\Users\isabe\AppData\Local\Temp\ipykernel_14616\20939837.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.




In [11]:
nta['uace_code'].fillna(nta['uace_code'].mode()[0], inplace=True)
nta['uza_name'].fillna('Unknown', inplace=True)

C:\Users\isabe\AppData\Local\Temp\ipykernel_14616\77905176.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


C:\Users\isabe\AppData\Local\Temp\ipykernel_14616\77905176.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)',

In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV

In [13]:
X = nta.drop(columns=['profit']) 
y = nta['profit']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [15]:
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object', 'category']).columns

In [16]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),      
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)  
    ]
)

In [17]:
ridge_pipeline = Pipeline([
    ('preprocessor', preprocessor),  
    ('ridge', Ridge(alpha=1.0))      
])

In [18]:
lasso_pipeline = Pipeline([
    ('preprocessor', preprocessor),  
    ('lasso', Lasso(alpha=0.1))     
])

In [19]:
ridge_scores = cross_val_score(ridge_pipeline, X_train, y_train, cv=5, scoring='r2')
print(f"Ridge Regression Cross-Validation R^2 Scores: {ridge_scores}")
print(f"Mean R^2 Score for Ridge: {ridge_scores.mean()}")

Ridge Regression Cross-Validation R^2 Scores: [ 0.90690411  0.73249554  0.98536959 -7.45008743  0.98578068]
Mean R^2 Score for Ridge: -0.7679075022665522


In [20]:
param_grid_ridge = {'ridge__alpha': [0.01, 0.1, 1.0, 10.0, 100.0]}
param_grid_lasso = {'lasso__alpha': [0.01, 0.1, 1.0, 10.0, 100.0]}

In [21]:
ridge_grid = GridSearchCV(ridge_pipeline, param_grid_ridge, cv=5, scoring='r2')
ridge_grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         Index(['ntd_id', 'report_year', 'uace_code', 'primary_uza_population', 'voms',
       'general_fund', 'income_tax', 'sales_tax', 'property_tax', 'fuel_tax',
       'other_tax', 'tolls', 'other_funds', 'reduced_funds', 'total_funding',
       'agency_voms', 'mode_voms', 'v...
       'reduced_reporter_expenses', 'total_expenses',
       'purchased_transportation_reported_separately'],
      dtype='object')),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         Index(['agency', 'city', 'state', 'organization_type', 'reporter_type',
       'uza_name', 'mode', 'mode_name', 'tos'],
      dtype='object'))])),
                                       ('ridge', Ridge())]),
             param_grid={'ridge__alpha': [0.01, 0.1, 1.0, 10.0, 100.0]},
             scoring='r2')

In [22]:
print(f"Best Ridge alpha: {ridge_grid.best_params_['ridge__alpha']}")
print(f"Best Ridge R^2: {ridge_grid.best_score_}")

Best Ridge alpha: 10.0
Best Ridge R^2: 0.7901376814478798


In [23]:
X_train

,agency,city,state,ntd_id,organization_type,reporter_type,report_year,uace_code,uza_name,primary_uza_population,...,mode_name,tos,mode_voms,vehicle_operations,vehicle_maintenance,facility_maintenance,general_administration,reduced_reporter_expenses,total_expenses,purchased_transportation_reported_separately
24,City of Long Beach,Long Beach,NY,20006,"City, County or Local Government Unit or Depar...",Full Reporter,2023,63217.0,"New York--Jersey City--Newark, NY--NJ",1.942645e+07,...,Bus,DO,5,2078171,608546,37830,271171,0,2995718,0
80,"Oswego County, dba: Oswego Public Transportation",Oswego,NY,20942,"City, County or Local Government Unit or Depar...",Rural Reporter,2023,63217.0,Unknown,9.184145e+06,...,Demand Response,PT,2,0,0,0,0,567894,567894,0
114,"Town of Highlands , dba: Town of Highlands Dia...",Highland Falls,NY,20182,"City, County or Local Government Unit or Depar...",Reduced Reporter,2023,71803.0,"Poughkeepsie--Newburgh, NY",3.147660e+05,...,Demand Response,DO,1,0,0,0,0,78911,78911,0
51,"Fulton County, dba: Brown Coach Commuter",Johnstown,NY,20964,"City, County or Local Government Unit or Depar...",Rural Reporter,2023,63217.0,Unknown,9.184145e+06,...,Bus,PT,1,0,0,0,0,313557,313557,0
15,"Chemung County, dba: C TRAN",Elmira,NY,20005,"City, County or Local Government Unit or Depar...",Reduced Reporter,2023,27118.0,"Elmira, NY",6.246800e+04,...,Demand Response,PT,5,0,0,0,0,673047,673047,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,New York City Economic Development Corporation,New York,NY,22930,Private-Non-Profit Corporation,Full Reporter,2023,63217.0,"New York--Jersey City--Newark, NY--NJ",1.942645e+07,...,Ferryboat,PT,25,33555017,9580494,5614641,20379962,0,69130114,0
106,"Suffolk County , dba: Suffolk County Transit",Yaphank,NY,20072,"City, County or Local Government Unit or Depar...",Full Reporter,2023,63217.0,"New York--Jersey City--Newark, NY--NJ",1.942645e+07,...,Bus,PT,124,33087677,9997644,2707232,11470269,0,57262822,0
14,"Chemung County, dba: C TRAN",Elmira,NY,20005,"City, County or Local Government Unit or Depar...",Reduced Reporter,2023,27118.0,"Elmira, NY",6.246800e+04,...,Bus,PT,19,0,0,0,0,5445565,5445565,0
92,RTS Wyoming,Rochester,NY,20980,Independent Public Agency or Authority of Tran...,Rural Reporter,2023,63217.0,Unknown,9.184145e+06,...,Bus,DO,14,0,0,0,0,1806846,1806846,0
